<a href="https://colab.research.google.com/github/endintro/wordman/blob/master/wind_helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xlsxwriter

In [95]:
# 1.导入pandas模块
import pandas as pd
import numpy as np

# 2.定义常量
dir = '/content/drive/MyDrive/Colab Notebooks/data/20211020/'
fn0 = '人保产品.xlsx'
sn0 = 'Sheet1'
fn1 = '债券型1020.xlsx'
sn1 = '万得'
fn2 = '混合型1020.xlsx'
sn2 = '万得'
fn3 = '股票型1020.xlsx'
sn3 = '万得'
fn4 = '货币类1020.xlsx'
sn4 = '万得'
fn5 = 'result.xlsx'
sn5 = 'Sheet1'

# 3.读取excel的某一个sheet
df0 = pd.read_excel(dir+fn0, sheet_name=sn0, names=['类型','证券代码','证券简称','全年目标收益率','备注'])

In [4]:
def getdf( fn, sn, cflag = False):
  # 3.读取excel的某一个sheet
  df1 = pd.read_excel(dir+fn, sheet_name=sn, names=['证券代码','证券简称','今年以来收益率','今年以来最大回撤'])

  # 4.删除有空值的行
  df1 = df1.dropna()

  # 5.取小数点后2位，如果是货币则保留每万份收益的位数
  if cflag:
    df1 = df1.round({'今年以来收益率':2})
  else:
    df1 = df1.round({'今年以来收益率':2,'今年以来最大回撤':2})

  # 6.按列排序
  df1.sort_values(['今年以来收益率'],ascending=[False])
  df1['num']=df1['今年以来收益率'].rank(ascending=0,method='first')

  # 7.增加分位列
  df1['percent'] = df1['num']*100/(df1.shape[0])
  df1 = df1.round({'percent':0})
  df1['rank'] = df1['num'].map(int).map(str)+'/'+str(df1.shape[0])

  # 8.合并数据
  df = pd.merge(df0, df1, on='证券代码')

  return df

In [5]:
# 债券
df_bund = getdf(fn1, sn1)
# 混合
df_mix = getdf(fn2, sn2)
# 权益
df_share = getdf(fn3, sn3)
# 货币
df_currency = getdf(fn4, sn4, True)

In [94]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(dir+fn5, engine='xlsxwriter')
workbook = writer.book

# 设置样式
format_red_border = workbook.add_format({'bg_color': '#f2d4d1', 'border': 1})
format_yellow_border = workbook.add_format({'bg_color': '#FFFF99', 'border': 1})
format_green_border = workbook.add_format({'bg_color': '#EBF1DE', 'border': 1})
format_border = workbook.add_format({'border': 1})
format_title = workbook.add_format()
format_title.set_font_size(14)
format_ps = workbook.add_format()
format_ps.set_font_size(9)
format_header_border = workbook.add_format({'align': 'center','border': 1})
format_middle_border = workbook.add_format({'align': 'vcenter','border': 1})
format_middle_border.set_align('center')



# 分位图例
title = pd.DataFrame({'Data': ['前30%', '30%-50%', '后50%']})
title.to_excel(writer, sheet_name=sn5, header=None, index=False, startcol=4,startrow=0)


worksheet = writer.sheets[sn5]

worksheet.conditional_format(0,4,0,4, {'type': 'cell','criteria': '!=','value': 0,'format': format_red_border})
worksheet.conditional_format(1,4,1,4, {'type': 'cell','criteria': '!=','value': 0,'format': format_yellow_border})
worksheet.conditional_format(2,4,2,4, {'type': 'cell','criteria': '!=','value': 0,'format': format_green_border})



# 1、净值型产品 ################################################################################
worksheet.write(3, 0, '1、净值型产品', format_title)

# 表头
worksheet.write_row(4, 0,  ['产品类型','产品名称','收益率分位数（%）','收益率-今年以来（%）','最大回撤-今年以来'],format_header_border)
current_row = 5

# 纯债 数据
bund = df_bund.query('类型=="纯债"').sort_values(['今年以来收益率'],ascending=[False]).loc[:,['证券简称_x','percent','今年以来收益率','今年以来最大回撤']]
worksheet.merge_range(current_row,0,current_row + bund.shape[0]-1,0, "纯债固收型", format_middle_border)
bund.to_excel(writer, sheet_name=sn5, header=None, index=False, startcol=1,startrow=current_row)

worksheet.conditional_format(current_row,1,(current_row + bund.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=30','format': format_red_border})
worksheet.conditional_format(current_row,1,(current_row + bund.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=50','format': format_yellow_border})
worksheet.conditional_format(current_row,1,(current_row + bund.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=100','format': format_green_border})

current_row = current_row + bund.shape[0] + 1

# 固收+ 数据
fixed = df_bund.query('类型=="固收+"').sort_values(['今年以来收益率'],ascending=[False]).loc[:,['证券简称_x','percent','今年以来收益率','今年以来最大回撤']]
worksheet.merge_range(current_row,0,current_row + fixed.shape[0]-1,0, "固收+", format_middle_border)
fixed.to_excel(writer, sheet_name=sn5, header=None, index=False, startcol=1,startrow=current_row)

worksheet.conditional_format(current_row,1,(current_row + fixed.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=30','format': format_red_border})
worksheet.conditional_format(current_row,1,(current_row + fixed.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=50','format': format_yellow_border})
worksheet.conditional_format(current_row,1,(current_row + fixed.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=100','format': format_green_border})

current_row = current_row + fixed.shape[0] + 1

# 混合 数据
mix = df_mix.query('类型=="混合"').sort_values(['今年以来收益率'],ascending=[False]).loc[:,['证券简称_x','percent','今年以来收益率','今年以来最大回撤']]

if mix.shape[0]>1:
  worksheet.merge_range(current_row,0,current_row + mix.shape[0]-1,0, "混合型", format_middle_border)
else:
  worksheet.write(current_row, 0, "混合型", format_middle_border)

mix.to_excel(writer, sheet_name=sn5, header=None, index=False, startcol=1,startrow=current_row)

worksheet.conditional_format(current_row,1,(current_row + mix.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=30','format': format_red_border})
worksheet.conditional_format(current_row,1,(current_row + mix.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=50','format': format_yellow_border})
worksheet.conditional_format(current_row,1,(current_row + mix.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=100','format': format_green_border})

current_row = current_row + mix.shape[0] + 1

# 权益 数据
share = df_share.query('类型=="权益"').sort_values(['今年以来收益率'],ascending=[False]).loc[:,['证券简称_x','percent','今年以来收益率','今年以来最大回撤']]

if share.shape[0]>1:
  worksheet.merge_range(current_row,0,current_row + share.shape[0]-1,0, "权益型", format_middle_border)
else:
  worksheet.write(current_row, 0, "权益型", format_middle_border)

share.to_excel(writer, sheet_name=sn5, header=None, index=False, startcol=1,startrow=current_row)

worksheet.conditional_format(current_row,1,(current_row + share.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=30','format': format_red_border})
worksheet.conditional_format(current_row,1,(current_row + share.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=50','format': format_yellow_border})
worksheet.conditional_format(current_row,1,(current_row + share.shape[0]-1),4, {'type': 'formula','criteria': '=INDIRECT("C"&ROW())<=100','format': format_green_border})

current_row = current_row + share.shape[0]

# 备注
ps = ['注1：数据统计截至2021年10月20日','注2：数据样本选取WIND披露且有净值数据的各类保险资管产品','注3：“纯债固收型”、“固收+”对标债券型保险资管产品，“混合型”对标混合型保险资管产品，“权益型”对标股票型保险资管产品']
worksheet.write_column(current_row, 0, ps, format_ps)

current_row = current_row + 3 + 3




# 2、净值型产品（绝对收益策略） ################################################################################
worksheet.write(current_row, 0, '2、净值型产品（绝对收益策略）', format_title)
current_row = current_row + 1

# 表头
worksheet.write_row(current_row, 0,  ['产品类型','产品名称','全年目标收益率(%)','收益率-今年以来（%）','最大回撤-今年以来','备注'], format_header_border)
current_row = current_row + 1

# 绝对收益 数据
worksheet.write(current_row, 0, "固收+", format_border)
worksheet.write(current_row+1, 0, "（权益配置区间0-5%）", format_border)

absolute = df_bund.query('类型=="绝对收益"').sort_values(['今年以来收益率'],ascending=[False]).loc[:,['证券简称_x','全年目标收益率','今年以来收益率','今年以来最大回撤','备注']]
worksheet.merge_range(current_row,0,current_row + absolute.shape[0]-1,0, "固收+"+chr(10)+"（权益配置区间0-5%）", format_middle_border)
absolute.to_excel(writer, sheet_name=sn5, header=None, index=False, startcol=1,startrow=current_row)
worksheet.conditional_format(current_row,1,(current_row + absolute.shape[0]-1),5, {'type': 'cell','criteria': '!=','value': 99999,'format': format_border})

current_row = current_row + absolute.shape[0]

# 备注
ps = ['注1：数据统计截至2021年10月20日','注2：数据样本选取WIND披露且有净值数据的债券型保险资管产品']
worksheet.write_column(current_row, 0, ps, format_ps)

current_row = current_row + 3 + 2




# 3、 货币型产品 ################################################################################
worksheet.write(current_row, 0, '3、 货币型产品', format_title)
current_row = current_row + 1

# 表头
worksheet.write_row(current_row, 0,  ['产品类型','产品名称','收益率分位数','七日年化收益（%）','每万份收益'], format_header_border)
current_row = current_row + 1

# 绝对收益 数据
worksheet.write(current_row, 0, "货币型", format_middle_border)
currency = df_currency.query('类型=="货币"').sort_values(['今年以来收益率'],ascending=[False]).loc[:,['证券简称_x','rank','今年以来收益率','今年以来最大回撤']]
currency.to_excel(writer, sheet_name=sn5, header=None, index=False, startcol=1,startrow=current_row)
worksheet.conditional_format(current_row,1,(current_row + currency.shape[0]-1),4, {'type': 'cell','criteria': '!=','value': 99999,'format': format_green_border})


current_row = current_row + currency.shape[0]

# 备注
ps = ['注1：数据统计截至2021年10月20日','注2：数据样本选取WIND披露且有收益数据的使用摊余成本法估值的货币类保险资']
worksheet.write_column(current_row, 0, ps, format_ps)

current_row = current_row + 3 + 3




# 设置列宽度
worksheet.set_column('A:A', 20)
worksheet.set_column('B:B', 30)
worksheet.set_column('C:C', 20)
worksheet.set_column('D:D', 22)
worksheet.set_column('E:E', 20)
worksheet.set_column('F:F', 30)

writer.save()